<a href="https://colab.research.google.com/github/NohaWaly/convolutional-neural-networks-keras/blob/master/Problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
! git clone https://github.com/ardamavi/Sign-Language-Digits-Dataset.git

fatal: destination path 'Sign-Language-Digits-Dataset' already exists and is not an empty directory.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
#cv2 to read image, show and more
import cv2
import os
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from keras import layers
from keras import models as ml
from keras import optimizers
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from numpy import argmax

DataDir = "Sign-Language-Digits-Dataset/Dataset"
ImgSize = 100

Categories = ["0", "1","2","3", "4","5","6", "7","8","9"]
data = []
#iterate over 10 folders
for category in Categories:  
    # create path to each gesture 
    path = os.path.join(DataDir,category) 
    classNum = Categories.index(category)
    #print(path)
    # iterate over each image per each gesture 
    for img in os.listdir(path): 
        print(img)
        # convert to array
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR) 
        # resize data
        resize_array = cv2.resize(img_array, (ImgSize, ImgSize))
        #append to data
        data.append([resize_array, classNum])
        # graph it
        #plt.imshow(img_array)
        # display!   
        #plt.show()  


In [0]:
#shuffle data (msh kol class ykon wara b3d)
random.shuffle(data)  
#print(data)


In [0]:
X = []
y = []

for features, label in data:
   X.append(features)
   y.append(label)
#to enter the model it should be np array
X = np.array(X)
y = np.array(y,dtype=int)

#hot incoder
y = to_categorical(y)

print(X)
print(y)

In [0]:
#normalization of data
#get mean of index 3 of shape =>3 layers
mean_array = np.mean(X, axis=3)
print(mean_array.shape)
mean_array = np.expand_dims(mean_array, axis=3)
print(mean_array.shape)
Xnormalized = (X - mean_array)/255.0
print(Xnormalized)

In [0]:
Xnormalized.shape

In [0]:
       
#spliting training & test sets
X_train, X_test, y_train, y_test = train_test_split(Xnormalized,y,test_size=0.2, random_state=7)

In [0]:

def create_cnn_model1():
    model = ml.Sequential()
    #32 filters, thier size 3*3
    model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(100, 100, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer = tf.train.AdamOptimizer(),metrics=['acc'])
    return model

In [0]:
def create_cnn_model2():
    model = ml.Sequential()
    #32 filters, thier size 3*3
    model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(100, 100, 3)))
    model.add(layers.AveragePooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.AveragePooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.AveragePooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer = tf.train.AdamOptimizer(),metrics=['acc'])
    return model

In [0]:
def create_cnn_model3():
    model = ml.Sequential()
    #32 filters, thier size 3*3
    model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(100, 100, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer = tf.train.AdamOptimizer(),metrics=['acc'])
    return model

In [0]:
def create_cnn_model4():
    model = ml.Sequential()
    #32 filters, thier size 3*3
    model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(100, 100, 3)))
    model.add(layers.AveragePooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.AveragePooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer = tf.train.AdamOptimizer(),metrics=['acc'])
    return model

In [90]:
n_split = 4
val_loss = np.zeros(n_split)
val_acc = np.zeros(n_split)

k_Fold = KFold(n_splits=4)
modeling = [create_cnn_model1,create_cnn_model2,create_cnn_model3,create_cnn_model4]
for x in range(4):
  i=0
  for train, validate in k_Fold.split(X_train, y_train):
    model = modeling[x]()
    model.fit(X_train[train], y_train[train], epochs=50)
    val_loss[i], val_acc[i] = model.evaluate(X_train[validate],y_train[validate])
    model.summary()
    i+=1
  print("Average Loss and Accuracy in Architecture" +str(x+1))
  print(val_loss.mean())
  print(val_acc.mean()) 
    #evaluate the testing part
  print("Evaluate testing part")
  y_test = to_categorical(y_test)
  model.evaluate(X_test, y_test)
  #get predicted labels
  y_predict = model.predict_classes(X_test)
  #reverse from categorial 
  y_test = np.argmax(y_test, axis=1, out=None)
  #evaluate model using precision, fscore, & recall
  print(classification_report(y_test, y_predict, target_names=Categories))

Epoch 1/50
1236/1236 [==============================] - 6s 5ms/step - loss: 1.6975 - acc: 0.4021
Epoch 2/50
1236/1236 [==============================] - 0s 374us/step - loss: 0.6964 - acc: 0.7767
Epoch 3/50
1236/1236 [==============================] - 0s 371us/step - loss: 0.4803 - acc: 0.8463
Epoch 4/50
1236/1236 [==============================] - 0s 379us/step - loss: 0.3733 - acc: 0.8940
Epoch 5/50
1236/1236 [==============================] - 0s 364us/step - loss: 0.2619 - acc: 0.9183
Epoch 6/50
1236/1236 [==============================] - 0s 381us/step - loss: 0.1891 - acc: 0.9458
Epoch 7/50
1236/1236 [==============================] - 0s 367us/step - loss: 0.1260 - acc: 0.9636
Epoch 8/50
1236/1236 [==============================] - 0s 393us/step - loss: 0.0800 - acc: 0.9765
Epoch 9/50
1236/1236 [==============================] - 0s 382us/step - loss: 0.0633 - acc: 0.9806
Epoch 10/50
1236/1236 [==============================] - 0s 365us/step - loss: 0.0544 - acc: 0.9822
Epoch 11/50